In [55]:
import pandas as pd
import json
from shapely.geometry import shape, GeometryCollection, Point
from shapely import wkt
import geopandas as gpd
from geopy.geocoders import Nominatim
geocode = Nominatim(user_agent="walter.tyrna@gmail.com")
from tqdm import tqdm

In [2]:
data_21_09 = pd.read_csv('202109-capitalbikeshare-tripdata.csv')


In [3]:
data_21_09.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,CA46A1E7C19A98DD,classic_bike,2021-09-15 18:19:28,2021-09-15 18:31:26,10th & Florida Ave NW,31120.0,Thomas Circle,31241.0,38.920387,-77.025672,38.905900,-77.032500,member
1,57F285E99CC0C4E5,classic_bike,2021-09-19 13:58:37,2021-09-19 14:15:22,10th & Florida Ave NW,31120.0,3rd & M St NE,31627.0,38.920387,-77.025672,38.905470,-77.002130,member
2,3F7C1AC5CF515183,classic_bike,2021-09-24 21:20:15,2021-09-24 21:23:18,10th & Florida Ave NW,31120.0,7th & T St NW,31109.0,38.920387,-77.025672,38.915500,-77.022200,member
3,F356866355DBD612,classic_bike,2021-09-30 20:58:35,2021-09-30 21:10:32,10th & Florida Ave NW,31120.0,4th St & K St NW,31642.0,38.920387,-77.025672,38.902406,-77.016006,member
4,35BB7A3DCF6E9BEB,docked_bike,2021-09-28 21:42:43,2021-09-28 21:54:12,10th & Florida Ave NW,31120.0,15th & Harvard St NW,31103.0,38.920387,-77.025672,38.926124,-77.035421,casual


In [4]:
with open('Neighborhood_Clusters.geojson', 'r') as f:
    js = json.load(f)


In [5]:
js

{'type': 'FeatureCollection',
 'name': 'Neighborhood_Clusters',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 1,
    'WEB_URL': 'http://planning.dc.gov/',
    'NAME': 'Cluster 39',
    'NBH_NAMES': 'Congress Heights, Bellevue, Washington Highlands',
    'Shape_Length': 10711.668009757177,
    'Shape_Area': 4886462.547938361,
    'TYPE': 'Original'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-76.99401890037231, 38.84519662346874],
      [-76.99470784344783, 38.8455876232067],
      [-76.9956363910663, 38.846156906271204],
      [-76.9962240974533, 38.84650726509601],
      [-76.99621095207603, 38.846699592666404],
      [-76.99609263072001, 38.84697723008251],
      [-76.99604124180318, 38.84723405902782],
      [-76.99602486552119, 38.8475636773414],
      [-76.9960104399866, 38.84797536146929],
      [-76.99600351570035, 38.848197149034306],
      [-76.9959949455402, 3

In [6]:
stations1 = data_21_09[['start_station_name', 'start_lng', 'start_lat']].copy()
stations1 = stations1.rename(columns={"start_station_name": "station_name"})

stations2 = data_21_09[['end_station_name', 'start_lng', 'start_lat']].copy()
stations2 = stations2.rename(columns={"end_station_name": "station_name"})

stations = pd.concat([stations1, stations2])

In [7]:
stations1

,station_name,start_lng,start_lat
0,10th & Florida Ave NW,-77.025672,38.920387
1,10th & Florida Ave NW,-77.025672,38.920387
2,10th & Florida Ave NW,-77.025672,38.920387
3,10th & Florida Ave NW,-77.025672,38.920387
4,10th & Florida Ave NW,-77.025672,38.920387
...,...,...,...
309192,5th & K St NW,-77.019027,38.903040
309193,Commonwealth Ave & Oak St,-77.061713,38.812591
309194,5th & K St NW,-77.019027,38.903040
309195,5th & K St NW,-77.019027,38.903040


In [8]:
stations = stations.drop_duplicates(subset=['station_name']).reset_index()

In [9]:
stations = stations.drop(['index'], axis=1)

In [10]:
stations['start_lat'] = stations['start_lat'].astype(str)
stations['start_lng'] = stations['start_lng'].astype(str)

In [11]:
stations['coord'] = stations['start_lng']+" "+stations['start_lat']

In [12]:
stations

,station_name,start_lng,start_lat,coord
0,10th & Florida Ave NW,-77.025672,38.920387,-77.025672 38.920387
1,13th St & New York Ave NW,-77.029822,38.900283,-77.029822 38.900283
2,11th & C St SE,-76.991476,38.885908,-76.991476 38.885908
3,Reservoir Rd & 38th St NW,-77.07432264076488,38.91261359469192,-77.07432264076488 38.91261359469192
4,Rhode Island & Connecticut Ave NW,-77.03980207443237,38.905995699991145,-77.03980207443237 38.905995699991145
...,...,...,...,...
652,Connecticut Ave & Chevy Chase Lake Dr,-77.076986,38.994113,-77.076986 38.994113
653,5th & K St NW,-77.019027,38.90304,-77.019027 38.90304
654,Wilson Blvd & N Troy St,-77.082089,38.892556,-77.082089 38.892556
655,Commonwealth Ave & Oak St,-77.061715,38.812711,-77.061715 38.812711


In [13]:
points = []


In [14]:
for i in stations['coord']:
    new_string = "Point(" + i + ")"
    points.append(new_string)

In [15]:
stations['points'] = points

In [16]:
geoms = []


In [17]:
for index, row in stations.iterrows():
    try:
        value = row["points"]
        geoms.append(wkt.loads(value))
    except Exception:
        geoms.append(f'{index} [{len(value)}] {value!r}')

In [18]:
stations['geoms'] = geoms

In [19]:
stations['geoms'].describe()

count                                              657
unique                                             656
top       POINT (-77.04409494996071 38.87056104566178)
freq                                                 2
Name: geoms, dtype: object

In [20]:
neighborhood = []

In [21]:
for i in stations['geoms']:
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        #print(polygon)
        if polygon.contains(i):
            neighborhood.append((i, feature['properties'].get('NBH_NAMES')))


In [22]:
len(neighborhood)

332

In [23]:
neighborhood

[(<shapely.geometry.point.Point at 0x7fd518605760>,
  'Howard University, Le Droit Park, Cardozo/Shaw'),
 (<shapely.geometry.point.Point at 0x7fd52802dc10>,
  'Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street'),
 (<shapely.geometry.point.Point at 0x7fd57a9bd8e0>,
  'Capitol Hill, Lincoln Park'),
 (<shapely.geometry.point.Point at 0x7fd5186054c0>,
  'Georgetown, Burleith/Hillandale'),
 (<shapely.geometry.point.Point at 0x7fd518605be0>,
  'Dupont Circle, Connecticut Avenue/K Street'),
 (<shapely.geometry.point.Point at 0x7fd518605d00>,
  'Capitol Hill, Lincoln Park'),
 (<shapely.geometry.point.Point at 0x7fd530497d00>,
  'Edgewood, Bloomingdale, Truxton Circle, Eckington'),
 (<shapely.geometry.point.Point at 0x7fd5304978b0>,
  'Kalorama Heights, Adams Morgan, Lanier Heights'),
 (<shapely.geometry.point.Point at 0x7fd530497fa0>,
  'Friendship Heights, American University Park, Tenleytown'),
 (<shapely.geometry.point.Point at 0x7fd530497a60>,
  'Southwest Emplo

In [24]:
df_neighborhoods = pd.DataFrame(neighborhood, columns =['geoms', 'neighborhood'])

In [25]:
df_neighborhoods['geoms2'] = df_neighborhoods['geoms'].astype(str)
df_neighborhoods['neighborhood'] = df_neighborhoods['neighborhood'].astype(str)

In [26]:
stations['geoms2'] = stations['geoms'].astype(str)

In [27]:
stations2 = stations.merge(df_neighborhoods, on='geoms2', how='left')

In [28]:
stations2 = stations2.drop(columns=(['points','geoms_x','geoms2']), axis=1)

In [53]:
stations2['coord2'] = stations2['start_lat'] + ', ' + stations2['start_lng']

In [54]:
stations2

,station_name,start_lng,start_lat,coord,geoms_y,neighborhood,coord2
0,10th & Florida Ave NW,-77.025672,38.920387,-77.025672 38.920387,POINT (-77.025672 38.920387),"Howard University, Le Droit Park, Cardozo/Shaw","38.920387, -77.025672"
1,13th St & New York Ave NW,-77.029822,38.900283,-77.029822 38.900283,POINT (-77.029822 38.900283),"Downtown, Chinatown, Penn Quarters, Mount Vern...","38.900283, -77.029822"
2,11th & C St SE,-76.991476,38.885908,-76.991476 38.885908,POINT (-76.99147600000001 38.885908),"Capitol Hill, Lincoln Park","38.885908, -76.991476"
3,Reservoir Rd & 38th St NW,-77.07432264076488,38.91261359469192,-77.07432264076488 38.91261359469192,POINT (-77.07432264076488 38.91261359469192),"Georgetown, Burleith/Hillandale","38.91261359469192, -77.07432264076488"
4,Rhode Island & Connecticut Ave NW,-77.03980207443237,38.905995699991145,-77.03980207443237 38.905995699991145,POINT (-77.03980207443237 38.90599569999114),"Dupont Circle, Connecticut Avenue/K Street","38.905995699991145, -77.03980207443237"
...,...,...,...,...,...,...,...
652,Connecticut Ave & Chevy Chase Lake Dr,-77.076986,38.994113,-77.076986 38.994113,NaN,NaN,"38.994113, -77.076986"
653,5th & K St NW,-77.019027,38.90304,-77.019027 38.90304,POINT (-77.01902699999999 38.90304),"Downtown, Chinatown, Penn Quarters, Mount Vern...","38.90304, -77.019027"
654,Wilson Blvd & N Troy St,-77.082089,38.892556,-77.082089 38.892556,NaN,NaN,"38.892556, -77.082089"
655,Commonwealth Ave & Oak St,-77.061715,38.812711,-77.061715 38.812711,NaN,NaN,"38.812711, -77.061715"


In [61]:
zipcodes = []

In [62]:
for i in tqdm(stations2.coord2):
    location = geocode.reverse(i)
    zipcode = location.raw['address']['postcode']
    zipcodes.append(zipcode)

100%|██████████| 657/657 [05:28<00:00,  2.00it/s]


In [63]:
zipcodes

['20060',
 '20005',
 '20003',
 '20007',
 '20036',
 '20003',
 '22205',
 '20260',
 '22305',
 '20009',
 '22230',
 '22206',
 '22217',
 '20016',
 '22314',
 '22205',
 '20202',
 '22201',
 '20011',
 '22314',
 '20016',
 '20002',
 '20018',
 '20012',
 '22204-5360',
 '20910',
 '20003',
 '20904',
 '20007',
 '22204',
 '20060',
 '20024',
 '20002',
 '20036-5305',
 '20009-5540',
 '22209',
 '22314',
 '20002',
 '22202',
 '22202',
 '20814',
 '22202',
 '20018',
 '22209',
 '20057',
 '20590',
 '20781',
 '20002',
 '20010',
 '20001',
 '22202',
 '22206',
 '20190',
 '22202',
 '20015',
 '22814',
 '20560',
 '20857',
 '20001',
 '22203',
 '20020',
 '20009',
 '20401',
 '20250',
 '22205',
 '20002',
 '20911',
 '20011',
 '22205',
 '20902',
 '20566',
 '20009',
 '20020',
 '20002',
 '20009',
 '20549',
 '20020',
 '20053',
 '20002',
 '20005',
 '20036',
 '20009',
 '22211',
 '20010',
 '20005',
 '20064',
 '20020',
 '20009-5540',
 '20010',
 '22204',
 '22305',
 '20007',
 '20011',
 '22314',
 '22201',
 '20852',
 '20912',
 '20003',


In [64]:
stations2['zipcodes'] = zipcodes

In [65]:
stations2.to_csv('station_location_info.csv',index=False)

In [66]:
stations2

,station_name,start_lng,start_lat,coord,geoms_y,neighborhood,coord2,zipcodes
0,10th & Florida Ave NW,-77.025672,38.920387,-77.025672 38.920387,POINT (-77.025672 38.920387),"Howard University, Le Droit Park, Cardozo/Shaw","38.920387, -77.025672",20060
1,13th St & New York Ave NW,-77.029822,38.900283,-77.029822 38.900283,POINT (-77.029822 38.900283),"Downtown, Chinatown, Penn Quarters, Mount Vern...","38.900283, -77.029822",20005
2,11th & C St SE,-76.991476,38.885908,-76.991476 38.885908,POINT (-76.99147600000001 38.885908),"Capitol Hill, Lincoln Park","38.885908, -76.991476",20003
3,Reservoir Rd & 38th St NW,-77.07432264076488,38.91261359469192,-77.07432264076488 38.91261359469192,POINT (-77.07432264076488 38.91261359469192),"Georgetown, Burleith/Hillandale","38.91261359469192, -77.07432264076488",20007
4,Rhode Island & Connecticut Ave NW,-77.03980207443237,38.905995699991145,-77.03980207443237 38.905995699991145,POINT (-77.03980207443237 38.90599569999114),"Dupont Circle, Connecticut Avenue/K Street","38.905995699991145, -77.03980207443237",20036
...,...,...,...,...,...,...,...,...
652,Connecticut Ave & Chevy Chase Lake Dr,-77.076986,38.994113,-77.076986 38.994113,NaN,NaN,"38.994113, -77.076986",20815
653,5th & K St NW,-77.019027,38.90304,-77.019027 38.90304,POINT (-77.01902699999999 38.90304),"Downtown, Chinatown, Penn Quarters, Mount Vern...","38.90304, -77.019027",20001
654,Wilson Blvd & N Troy St,-77.082089,38.892556,-77.082089 38.892556,NaN,NaN,"38.892556, -77.082089",22205
655,Commonwealth Ave & Oak St,-77.061715,38.812711,-77.061715 38.812711,NaN,NaN,"38.812711, -77.061715",22314
